In [1]:
print("Hello")

Hello


In [2]:
import os
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

In [3]:
load_dotenv()  # load environment variables from .env

# --- Environment Variable Checks ---
google_api_key_loaded = os.getenv("GOOGLE_API_KEY") is not None

print(f"Google API Key Loaded: {google_api_key_loaded}")

Google API Key Loaded: True


In [4]:
model = init_chat_model("gemini-2.0-flash", 
                        model_provider="google_genai",
                        temperature=0)
# model = init_chat_model("gemini-1.5-flash", model_provider="google_genai")

response = model.invoke("Hello, world!")
print(response)

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--caa20523-0367-4195-9b09-1a4193976046-0' usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}}


In [5]:
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [6]:
# Add memory
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()

# Structured Output
class WeatherResponse(BaseModel):
    conditions: str

In [7]:
agent = create_react_agent(
    # model="anthropic:claude-3-7-sonnet-latest",
    # model="gemini/gemini-pro",
    model=model,
    tools=[get_weather],
    checkpointer=checkpointer,
    response_format=WeatherResponse
)

In [9]:
# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
print(sf_response)
print("****** structure response is ******\n", sf_response["structured_response"])


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='aab8d83c-7586-41be-b116-b4860b623ae5'), AIMessage(content='Could you please spell out the full city name?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6116c7cd-8f36-45d7-8366-f683d888791c-0', usage_metadata={'input_tokens': 20, 'output_tokens': 11, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='96880ff2-0b18-484a-88a4-9a061c503092'), AIMessage(content='Could you please provide the full city name, including the state or country? For example, "San Francisco, USA".', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0

In [11]:
# Run the agent
config = {"configurable": {"thread_id": "1"}}
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config,
    stream_mode="updates" 
):
    print(chunk)
# print("****** structure response is ******\n", sf_response["structured_response"])


{'agent': {'messages': [AIMessage(content='Please provide the complete city name, including the state and country, so I can retrieve the weather information.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9f253082-c4fe-43b3-ab1e-65d75ac0c556-0', usage_metadata={'input_tokens': 89, 'output_tokens': 22, 'total_tokens': 111, 'input_token_details': {'cache_read': 0}})]}}
{'generate_structured_response': {'structured_response': WeatherResponse(conditions='sunny')}}


In [12]:
# Run the agent
config = {"configurable": {"thread_id": "1"}}
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config,
    stream_mode="messages" 
):
    print("Token", token)
    print("Metadata", metadata)
    print("\n")

# print("****** structure response is ******\n", sf_response["structured_response"])


Token content='I cannot' additional_kwargs={} response_metadata={'safety_ratings': []} id='run--f873ba32-001b-4716-8815-73b5873ff05a' usage_metadata={'input_tokens': 150, 'output_tokens': 0, 'total_tokens': 150, 'input_token_details': {'cache_read': 0}}
Metadata {'thread_id': '1', 'langgraph_step': 17, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:5e230055-7b41-5053-f053-d40f51972f60', 'checkpoint_ns': 'agent:5e230055-7b41-5053-f053-d40f51972f60', 'ls_provider': 'google_genai', 'ls_model_name': 'gemini-2.0-flash', 'ls_model_type': 'chat', 'ls_temperature': 0.0}


Token content=" look up the weather for 'sf' because it is not a complete city name. Please" additional_kwargs={} response_metadata={'safety_ratings': []} id='run--f873ba32-001b-4716-8815-73b5873ff05a' usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0, 'input_token_details': {'cache_read': 0}}
Metadat

### summarize history of messages

In [15]:
from langmem.short_term import SummarizationNode
from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
from typing import Any

model = init_chat_model("gemini-2.0-flash", 
                        model_provider="google_genai",
                        temperature=0)
# model = init_chat_model("gemini-1.5-flash", model_provider="google_genai")

summarization_node = SummarizationNode( 
    token_counter=count_tokens_approximately,
    model=model,
    max_tokens=384,
    max_summary_tokens=128,
    output_messages_key="llm_input_messages",
)

class State(AgentState):
    # NOTE: we're adding this key to keep track of previous summary information
    # to make sure we're not summarizing on every LLM call
    context: dict[str, Any]  


checkpointer = InMemorySaver() 

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    pre_model_hook=summarization_node, 
    state_schema=State, 
    checkpointer=checkpointer,
)

AttributeError: module 'typing' has no attribute 'NotRequired'

### Multi Agent Support

#### Supervisor

![image](images\supervisor.png)

In [1]:
# pip install langgraph-supervisor


from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool


model = init_chat_model("gemini-2.0-flash", 
                        model_provider="google_genai",
                        temperature=0)

@tool
def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

@tool
def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

flight_assistant = create_react_agent(
    model=model,
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)

hotel_assistant = create_react_agent(
    model=model,
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    model=model,
    prompt=(
        "You manage a hotel booking assistant and a"
        "flight booking assistant. Assign work to them."
    )
).compile()

for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'supervisor': None}


{'flight_assistant': {'messages': [AIMessage(content='OK. I am now in flight booking mode.\n\nI can book a flight for you from BOS to JFK. Is that correct?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, name='flight_assistant', id='run--773df163-9acb-4638-824a-27687f9172f6-0', usage_metadata={'input_tokens': 63, 'output_tokens': 28, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}}), AIMessage(content='Transferring back to supervisor', additional_kwargs={}, response_metadata={'__is_handoff_back': True}, name='flight_assistant', tool_calls=[{'name': 'transfer_back_to_supervisor', 'args': {}, 'id': '7625b8ae-63f7-4b2d-ba50-be9a39fda915', 'type': 'tool_call'}]), ToolMessage(content='Successfully transferred back to supervisor', name='transfer_back_to_supervisor', tool_call_id='7625b8ae-63f7-4b2d-ba50-be9a39fda915')]}

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 50
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 47
}
]

#### Swarm

![image](images\swarm.png)

In [21]:
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_swarm, create_handoff_tool

transfer_to_hotel_assistant = create_handoff_tool(
    agent_name="hotel_assistant",
    description="Transfer user to the hotel-booking assistant.",
)
transfer_to_flight_assistant = create_handoff_tool(
    agent_name="flight_assistant",
    description="Transfer user to the flight-booking assistant.",
)

flight_assistant = create_react_agent(
    model=model,
    tools=[book_flight, transfer_to_hotel_assistant],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)
hotel_assistant = create_react_agent(
    model=model,
    tools=[book_hotel, transfer_to_flight_assistant],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

swarm = create_swarm(
    agents=[flight_assistant, hotel_assistant],
    default_active_agent="flight_assistant"
).compile()

for chunk in swarm.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'flight_assistant': {'messages': [HumanMessage(content='book a flight from BOS to JFK and a stay at McKittrick Hotel', additional_kwargs={}, response_metadata={}, id='a1b36c13-c65a-4bb4-8d3c-83f5af235d56'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'book_flight', 'arguments': '{"from_airport": "BOS", "to_airport": "JFK"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, name='flight_assistant', id='run--69115562-e77d-4310-a884-c58f3fb4287b-0', tool_calls=[{'name': 'book_flight', 'args': {'from_airport': 'BOS', 'to_airport': 'JFK'}, 'id': '7daa812a-cabb-4ca3-8ae5-43bbd17fc7c8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 11, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Successfully booked a flight from BOS to JFK.', name='book_flight', id='2d95d128-a61b-4ab0-9be7-cec421f49e08

In [22]:
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.types import Command

def create_handoff_tool(*, agent_name: str, description: str | None = None):
    name = f"transfer_to_{agent_name}"
    description = description or f"Transfer to {agent_name}"

    @tool(name, description=description)
    def handoff_tool(
        state: Annotated[MessagesState, InjectedState], 
        tool_call_id: Annotated[str, InjectedToolCallId],
    ) -> Command:
        tool_message = {
            "role": "tool",
            "content": f"Successfully transferred to {agent_name}",
            "name": name,
            "tool_call_id": tool_call_id,
        }
        return Command(  
            goto=agent_name,  
            update={"messages": state["messages"] + [tool_message]},  
            graph=Command.PARENT,  
        )
    return handoff_tool

# Handoffs
transfer_to_hotel_assistant = create_handoff_tool(
    agent_name="hotel_assistant",
    description="Transfer user to the hotel-booking assistant.",
)
transfer_to_flight_assistant = create_handoff_tool(
    agent_name="flight_assistant",
    description="Transfer user to the flight-booking assistant.",
)

# Simple agent tools
def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

# Define agents
flight_assistant = create_react_agent(
    model=model,
    tools=[book_flight, transfer_to_hotel_assistant],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)
hotel_assistant = create_react_agent(
    model=model,
    tools=[book_hotel, transfer_to_flight_assistant],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

# Define multi-agent graph
multi_agent_graph = (
    StateGraph(MessagesState)
    .add_node(flight_assistant)
    .add_node(hotel_assistant)
    .add_edge(START, "flight_assistant")
    .compile()
)

# Run the multi-agent graph
for chunk in multi_agent_graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'flight_assistant': {'messages': [HumanMessage(content='book a flight from BOS to JFK and a stay at McKittrick Hotel', additional_kwargs={}, response_metadata={}, id='e86b6c8f-c401-426d-8927-447d537edf28'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'book_flight', 'arguments': '{"from_airport": "BOS", "to_airport": "JFK"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, name='flight_assistant', id='run--8d7edea4-092e-481c-9baa-f8d0fb4aa56f-0', tool_calls=[{'name': 'book_flight', 'args': {'from_airport': 'BOS', 'to_airport': 'JFK'}, 'id': '19efa4d2-c685-4bcc-a4fa-6deeabab32c4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 11, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Successfully booked a flight from BOS to JFK.', name='book_flight', id='3d5dd5b1-1140-495f-bb28-63c24cc006a7